<a href="https://colab.research.google.com/github/ajaysingh-codes/farmworker-health-rag/blob/main/farmworker_health_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Farmworker Health Research RAG System

**A smart search system for scientific literature on farmworker health, chemical exposures, and occupational stressors**

Built with BM25 + Semantic Search | Interactive comparison interface | Optimized for health research papers

---



## 📦 Installing Required Libraries
This cell installs all the necessary Python packages for our RAG system:
- `sentence-transformers`: For creating semantic embeddings of text
- `bm25s`: For keyword-based search (BM25 algorithm)
- `pypdf2`: For extracting text from PDF files
- `pandas`: For data manipulation
- `numpy`: For numerical operations
- `joblib`: For saving/loading embeddings
- `ipywidgets`: For creating the interactive interface

In [1]:
!pip install sentence-transformers bm25s pypdf2 pandas numpy joblib ipywidgets -q
print("Libraries installed successfully")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

## Importing Libraries and Creating Workspace
This cell:
1. Imports all the libraries we'll use throughout the project
2. Creates a dedicated folder `/content/papers` for your PDF files
3. Sets up the basic environment for our RAG system

In [3]:
# Import all necessary libraries
import os
import pandas as pd
import numpy as np
import bm25s
import joblib
from sentence_transformers import SentenceTransformer
from IPython.display import display, Markdown
import ipywidgets as widgets
from PyPDF2 import PdfReader
from datetime import datetime

# Create a folder for your PDFs
pdf_folder = "/content/papers"
if not os.path.exists(pdf_folder):
    os.makedirs(pdf_folder)
    print(f"✅ Created folder: {pdf_folder}")
else:
    print(f"📁 Folder already exists: {pdf_folder}")

✅ Created folder: /content/papers


## Upload Your Research Papers
Upload your 5 PDF papers about farmworker health:
- Click 'Choose Files' to select your PDFs
- Papers will be moved to the `papers` folder
- You'll see confirmation for each uploaded file

In [4]:
from google.colab import files

print("Click 'Choose Files' below to upload your 5 PDFs:\n")

uploaded = files.upload()

# Move uploaded files to the papers folder
for filename in uploaded.keys():
    destination = os.path.join(pdf_folder, filename)
    os.rename(filename, destination)
    print(f"✅ Uploaded: {filename}")

# Verify and list all PDFs in the folder
pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]
print(f"📊 Total PDFs in folder: {len(pdf_files)}")
for i, pdf in enumerate(pdf_files, 1):
    print(f"  {i}. {pdf}")

Click 'Choose Files' below to upload your 5 PDFs:



Saving A_more_than_four-fold_sex-specific_difference_of_autism_spectrum_disorders_and_the_possible_contribu.pdf to A_more_than_four-fold_sex-specific_difference_of_autism_spectrum_disorders_and_the_possible_contribu.pdf
Saving Adverse_childhood_experiences_and_its_association_with_emotional_and_behavioral_problems_in_US_child.pdf to Adverse_childhood_experiences_and_its_association_with_emotional_and_behavioral_problems_in_US_child.pdf
Saving Agricultural_exposures_and_risk_of_childhood_neuroblastoma_a_systematic_review_and_meta-analysis.pdf to Agricultural_exposures_and_risk_of_childhood_neuroblastoma_a_systematic_review_and_meta-analysis.pdf
Saving Autism_Spectrum_Disorder_and_Prenatal_or_Early_Life_Exposure_to_Pesticides_A_Short_Review.pdf to Autism_Spectrum_Disorder_and_Prenatal_or_Early_Life_Exposure_to_Pesticides_A_Short_Review.pdf
Saving Exposure_to_pesticides_and_childhood_leukemia_risk_A_systematic_review_and_meta-analysis.pdf to Exposure_to_pesticides_and_childhood_leukemia_r

## Chunking Function with Overlap
Using 150-word chunks with 20% overlap for optimal context

In [5]:
from typing import List
import re

def get_chunks_fixed_size_with_overlap(text: str, chunk_size: int, overlap_fraction: float) -> List[str]:
  """
  Splits text into fixed-size chunks with overlap.

  Parameters:
  - text (str): The text to be split into chunks.
  - chunk_size (int): The desired size of each chunk.
  - overlap_fraction (float): The fraction of overlap between chunks (0.2 = 20% overlap)

  Returns:
  - List[str]: A list of text chunks where each chunk might overlap with its adjacent chunk.

  """
  # Split text into individual words
  text_words = text.split()

  # Calculate the number of words to overlap
  overlap_int = int(chunk_size * overlap_fraction)

  #Initialize a list to store resulting chunks
  chunks = []

  # Create chunks with overlap
  for i in range(0, len(text_words), chunk_size):
    # Include overlap from previous chunk
    chunk_words = text_words[max(i - overlap_int, 0): i + chunk_size]

    # Join words to form chunk
    chunk = " ".join(chunk_words)

    chunks.append(chunk)

  return chunks

## 🔍 Extract and Process PDFs
Extract text and create chunks from each paper

In [8]:
def extract_and_chunk_pdf(pdf_path, chunk_size=150, overlap=0.2):
    """Extract text from PDF and return chunks"""
    try:
        reader = PdfReader(pdf_path)
        full_text = ""

        # Extract all pages
        for page in reader.pages:
            text = page.extract_text()
            if text:
                full_text += text + " "

        # Clean text
        full_text = re.sub(r'\s+', ' ', full_text)
        full_text = re.sub(r'\n+', ' ', full_text)

        # Create chunks
        chunks = get_chunks_fixed_size_with_overlap(full_text, chunk_size, overlap)

        return chunks, len(reader.pages), True

    except Exception as e:
        print(f"❌ Error: {e}")
        return [], 0, False

# Process all PDFs and create dataset
PAPERS_DATA = []

for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_folder, pdf_file)
    paper_name = pdf_file.replace('.pdf', '')

    # Extract and chunk
    chunks, num_pages, success = extract_and_chunk_pdf(pdf_path)

    if success:
        for i, chunk in enumerate(chunks):

            PAPERS_DATA.append({
                'content': chunk, # Main text for search
                'source': paper_name, # Which paper it's from
                'id': f"{paper_name}_{i}" # Pos in paper
            })
    else:
        print(f"  ❌ Failed to process")

print(f"\n✅ Total chunks created: {len(PAPERS_DATA)}")


✅ Total chunks created: 261


## Step 3: Implement Retrieval functions
Build BM25 (keyword) and Semantic Search capabilities.

In [10]:
# Create corpus for searching (combining all text for each chunk)
corpus = [x['content'] for x in PAPERS_DATA]

print(f"Corpus ready with {len(corpus)} chunks")
print(f"From {len(set([x['source'] for x in PAPERS_DATA]))} papers")

Corpus ready with 261 chunks
From 5 papers
